In [6]:
%matplotlib inline

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.style.use('ggplot')

import os, sys, shutil
sys.path.append("..")
import pandas as pd

from lammpsrun import LAMMPS, Prism
from PBSJobSubmissionScript import PBS_Submitter, qstat_monitor
from utility_scripts import *

from ase import Atoms, units
from ase.visualize import view
from ase.visualize.plot import plot_atoms
from ase.io.trajectory import Trajectory
from ase.io import write, read
from ase.build import molecule, bcc110, bcc111
from ase.spacegroup import crystal
from ase.data import atomic_numbers, atomic_names, atomic_masses, covalent_radii

cx1_out = r"/rds/general/user/yx6015/home/cx1_out/"
ephemeral = r"/rds/general/user/yx6015/ephemeral/"
sim_path = r"/rds/general/user/yx6015/home/LAMMPS_Simulation/HPC_Jupyter/"
print(cx1_out + '\n' + sim_path)
potential = "fe_o_water_fe_o_c_full_water.reax"

/rds/general/user/yx6015/home/cx1_out/
/rds/general/user/yx6015/home/LAMMPS_Simulation/HPC_Jupyter/


# MKL/Multiprocessing Test

In [2]:
file_name = 'test'
lammps_traj_file = 'test.lammpstrj'
mol_file = file_name + ".extxyz"
ase_traj_file = file_name + ".traj"

In [3]:
%%prun -l 10
import sys
import time
sys.path.append("..")
from ase import Atoms, units
from ase.io.trajectory import Trajectory
from ase.io import write, read
from minimal_traj_conversion import LAMMPS, Prism
import mkl
mkl.set_num_threads(8)

_start = time.time()

calc = LAMMPS()
mol = read(mol_file)
mol.set_calculator(calc)
calc.atoms = mol
calc.prism = Prism(mol.get_cell())

print("Writing ASE trajectory to ", ase_traj_file)
calc.trajectory_out = Trajectory(ase_traj_file, 'w')

print("Reading LAMMPS Trajectory from", lammps_traj_file)
calc.read_lammps_trj(lammps_trj = lammps_traj_file, n_cpus=8)
calc.trajectory_out.close()

print("Took {} seconds".format(time.time() - _start))

Writing ASE trajectory to  test.traj
Reading LAMMPS Trajectory from test.lammpstrj
Took 11.05019497871399 seconds
 

         243177 function calls (234417 primitive calls) in 24.937 seconds

   Ordered by: internal time
   List reduced from 2374 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       29   10.332    0.356   10.332    0.356 {built-in method posix.waitpid}
      303    3.737    0.012    3.737    0.012 {method 'read' of '_io.FileIO' objects}
     2158    3.481    0.002    3.481    0.002 {built-in method posix.stat}
      303    2.277    0.008    6.014    0.020 <frozen importlib._bootstrap_external>:914(get_data)
       57    1.460    0.026    1.460    0.026 {built-in method _imp.create_dynamic}
       50    1.448    0.029    1.448    0.029 {built-in method posix.listdir}
        1    0.837    0.837    0.837    0.837 {method 'dot' of 'numpy.ndarray' objects}
      598    0.220    0.000    0.220    0.000 {built-in method io.open}
        4    0.209    0.052    0.209    0.052 linalg.py:486(inv)
      301    0.099    0.000    0.099    0.

In [2]:
%%prun -l 10
import sys
import time
sys.path.append("..")
from ase import Atoms, units
from ase.io.trajectory import Trajectory
from ase.io import write, read
from minimal_traj_conversion_queue import LAMMPS, Prism
import mkl
mkl.set_num_threads(8)

_start = time.time()

calc = LAMMPS()
mol = read(mol_file)
mol.set_calculator(calc)
calc.atoms = mol
calc.prism = Prism(mol.get_cell())

print("Writing ASE trajectory to ", ase_traj_file)
calc.trajectory_out = Trajectory(ase_traj_file, 'w')

print("Reading LAMMPS Trajectory from", lammps_traj_file)
calc.read_lammps_trj(lammps_trj = lammps_traj_file)
calc.trajectory_out.close()

print("Took {} seconds".format(time.time() - _start))

Writing ASE trajectory to  test.traj
Reading LAMMPS Trajectory from test.lammpstrj
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256

         275735 function calls (265203 primitive calls) in 7.766 seconds

   Ordered by: internal time
   List reduced from 2724 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       46    6.828    0.148    6.828    0.148 {built-in method posix.waitpid}
      598    0.122    0.000    0.122    0.000 {built-in method io.open}
     2496    0.099    0.000    0.099    0.000 {built-in method posix.stat}
      391    0.085    0.000    0.092    0.000 <frozen importlib._bootstrap_external>:914(get_data)
  942/917    0.067    0.000    0.125    0.000 {built-in method builtins.__build_class__}
      389    0.063    0.000    0.063    0.000 {built-in method marshal.loads}
       65    0.028    0.000    0.028    0.000 {built-in method _imp.create_dynamic}
       14    0.026    0.002    0.026    0.002 {built-in method numpy.dot}
      958    0.023    0.000    0.023    0.000 {method 'sub' of 're.Pattern' objects}
      642    0.017    0.000    0.0

In [2]:
%%prun -l 10
import sys
import time
sys.path.append("..")
from ase import Atoms, units
from ase.io.trajectory import Trajectory
from ase.io import write, read
from lammpsrun import LAMMPS, Prism
import mkl
mkl.set_num_threads(8)

_start = time.time()

calc = LAMMPS()
mol = read(mol_file)
mol.set_calculator(calc)
calc.atoms = mol
calc.prism = Prism(mol.get_cell())

print("Writing ASE trajectory to ", ase_traj_file+"1")
calc.trajectory_out = Trajectory(ase_traj_file+"1", 'w')

print("Reading LAMMPS Trajectory from", lammps_traj_file)
calc.read_lammps_trj(lammps_trj = lammps_traj_file)
calc.trajectory_out.close()

print("Took {} seconds".format(time.time() - _start))

Writing ASE trajectory to  test.traj1
Reading LAMMPS Trajectory from test.lammpstrj
Took 18.122143268585205 seconds
 

         11819424 function calls (11758968 primitive calls) in 18.265 seconds

   Ordered by: internal time
   List reduced from 2828 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      650    4.744    0.007    5.038    0.008 {method 'read' of 'pandas._libs.parsers.TextReader' objects}
     1303    2.100    0.002    2.100    0.002 {method 'tofile' of 'numpy.ndarray' objects}
      650    1.155    0.002    1.178    0.002 parsers.py:1830(__init__)
      651    0.777    0.001    1.209    0.002 symbols.py:61(symbols2numbers)
     3802    0.743    0.000    0.743    0.000 {built-in method posix.stat}
2346896/2343295    0.687    0.000    0.794    0.000 {built-in method builtins.next}
  3114720    0.587    0.000    0.816    0.000 {built-in method builtins.isinstance}
        1    0.541    0.541   17.982   17.982 lammpsrun.py:569(read_lammps_trj)
38497/37847    0.354    0.000    0.363    0.000 {built-in method numpy.array}
      650    0.

# Conversion Speed Test

In [2]:
aq_out = '2466596.cx1'
aq_name = 'Aq_efield_0MVcm'
out_dir = os.path.join(ephemeral, aq_out)
convert_scripts_dir = r"/rds/general/user/yx6015/home/LAMMPS_Simulation/HPC_Scripts/"

In [12]:
test_list = ['', '_nocopy', '_nocopy_nosend', '_fast']
script_list = ["convert_script{}.py".format(x) for x in test_list]
test_args = ['{}'.format(aq_name),
             '{} {}'.format(aq_name, os.path.join(out_dir, aq_name+'.lammpstrj')),
             '{} {} {}'.format(aq_name, os.path.join(out_dir, aq_name+'.lammpstrj'), os.path.join(out_dir, aq_name+'.traj1')),
             '{}'.format(aq_name)]
test_commands = [["python {} {}".format(x[0], x[1])] for x in zip(script_list, test_args)]

In [13]:
test_source_files = [[os.path.join(out_dir, "*extxyz"), 
                      os.path.join(convert_scripts_dir, x),
                      os.path.join(convert_scripts_dir, "lammpsrun.py")] for x in script_list]
test_source_files[0] += [os.path.join(out_dir, "*lammpstrj")]
test_source_files[3] += [os.path.join(out_dir, "*lammpstrj")]
test_names = ['convert{}'.format(x) for x in test_list]

In [14]:
convert_PBS = PBS_Submitter(job_names=test_names,
                            job_commands=test_commands,
                            modules=["mpi", "anaconda3/personal"],
                            walltime="24:00:00",
                            proc_nodes=1,
                            proc_cpus=8,
                            proc_mpiprocs=8, 
                            memory=32,
                            source_files=test_source_files)

In [15]:
convert_out, convert_err = convert_PBS.run()
print(convert_out)
print(test_names)

['2475004.cx1', '2475005.cx1', '2475006.cx1', '2475007.cx1', '2475008.cx1']
['convert', 'convert_nocopy', 'convert_nocopy_nosend', 'convert_fast', 'convert_nocopy_nosend_fast']


### Final results:
- *fast* is slightly faster (by ~9%)
- *nocopy* is faster than both *nosend* (by 30%) and *normal* (by ~44%)

### Conclusion
Use mkl acceleration and do not copy over lammpstrj